In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!pip install geemap

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# import geemap
import datetime
import numpy as np
import math

In [ ]:
area = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020_the_second_method/four')

In [ ]:
def maskS2clouds(image):
    qa = image.select('QA60');
    cloudBitMask = 1 << 10;
    cirrusBitMask = 1 << 11;
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0));
    return image.updateMask(mask);

def clip(image):
    image = image.clip(area);
    return image;

In [ ]:
start_time = "2020-02-01"
end_time = "2020-12-31"
day_gap = 16
start = ee.Date(start_time);
end = ee.Date(end_time);
day_number = end.difference(start, "day").floor()
image_number = day_number.divide(day_gap).floor()
print(image_number.getInfo())

In [ ]:
# 构建NDVI时间序列
def NDVI_timeMax(start_time,end_time,day_gap,image_number):

  def NDVI(image):
    nir = image.select("B8");
    red = image.select("B4");
    ndvi = nir.subtract(red).divide(nir.add(red));
    return image.select().addBands(ndvi);

  image_list = []
  NDVI_list = []

  for i in range(image_number.getInfo()):
      s_date = datetime.datetime.strptime(start_time, "%Y-%m-%d") + datetime.timedelta(days = (i)*day_gap)
      e_date = datetime.datetime.strptime(start_time, "%Y-%m-%d") + datetime.timedelta(days = (i+1)*day_gap)
      s_time = s_date.strftime("%Y-%m-%d");
      e_time = e_date.strftime("%Y-%m-%d");
      NDVI_image = ee.ImageCollection("COPERNICUS/S2_SR")\
                    .filterDate(s_time, e_time)\
                    .map(clip).filterBounds(area).map(NDVI).max().rename("NDVI").set("data",s_date)\
                    .set('system:footprint',area.geometry())
      ddiff = ee.Date(s_time).difference(ee.Date(start_time), 'day')
      new_NDVI_image = NDVI_image.addBands(ee.Image(1).toFloat().rename('constant'))\
                                .addBands(ee.Image(ddiff).toFloat().rename('t'))\
                                .addBands(ee.Image(ddiff).pow(ee.Image(2)).toFloat().rename('t2'))\
                                .addBands(ee.Image(ddiff).pow(ee.Image(3)).toFloat().rename('t3'))
      globals()["NDVI_image"+s_date.strftime("%Y%m%d")] = NDVI_image
      image_list.append(globals()["NDVI_image"+s_date.strftime("%Y%m%d")])
      NDVI_list.append("NDVI"+'{:0>2d}'.format(i))

  NDVI_col = ee.ImageCollection(image_list)
  NDVI_image = NDVI_col.toBands().rename(NDVI_list)
  return NDVI_col,NDVI_image

In [ ]:
NDVI = NDVI_timeMax(start_time,end_time,day_gap,image_number)
NDVI_col = NDVI[0]
NDVI_image = NDVI[1]

In [ ]:
Map = geemap.Map()
Map

In [ ]:
Map.addLayer(NDVI_image)

In [ ]:
corn = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020point/origin_point/corn').filterBounds(area)
cotton = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020point/origin_point/cotton').filterBounds(area)
forest = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020point/origin_point/nut').filterBounds(area)
wheat = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020point/origin_point/wheat').filterBounds(area)
# wheat_corn = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020point/origin_point/wheat_corn').filterBounds(area)
other = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020point/origin_point/other').filterBounds(area)

In [ ]:
corn_result = NDVI_image.sampleRegions(corn, scale=10, tileScale=8)
geemap.ee_export_vector_to_drive(corn_result, description="corn", folder="原始样本点NDVI时间序列", fileNamePrefix=None, fileFormat="csv")
cotton_result = NDVI_image.sampleRegions(cotton, scale=10, tileScale=8)
geemap.ee_export_vector_to_drive(cotton_result, description="cotton", folder="原始样本点NDVI时间序列", fileNamePrefix=None, fileFormat="csv")
forest_result = NDVI_image.sampleRegions(forest, scale=10, tileScale=8)
geemap.ee_export_vector_to_drive(forest_result, description="forest", folder="原始样本点NDVI时间序列", fileNamePrefix=None, fileFormat="csv")
wheat_result = NDVI_image.sampleRegions(wheat, scale=10, tileScale=8)
geemap.ee_export_vector_to_drive(wheat_result, description="wheat", folder="原始样本点NDVI时间序列", fileNamePrefix=None, fileFormat="csv")
# wheat_corn_result = NDVI_image.sampleRegions(wheat_corn, scale=10, tileScale=8)
# geemap.ee_export_vector_to_drive(wheat_corn_result, description="wheat_corn", folder="原始样本点NDVI时间序列", fileNamePrefix=None, fileFormat="csv")
other_result = NDVI_image.sampleRegions(other, scale=10, tileScale=8)
geemap.ee_export_vector_to_drive(other_result, description="other", folder="原始样本点NDVI时间序列", fileNamePrefix=None, fileFormat="csv")

In [ ]:
import csv
import os, sys
import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 打开矢量
csv_origin_dir = r"/content/gdrive/MyDrive/原始样本点NDVI时间序列"
os.chdir(csv_origin_dir)
CSV_list = glob.glob("*.csv")
print(CSV_list)
NDVI = []
i = 0
print(CSV_list[i])
with open(os.path.join(csv_origin_dir, CSV_list[i]), encoding='utf-8-sig') as f :
  for row in csv.reader(f, skipinitialspace=True):
    NDVI.append(row)
n = np.array(NDVI).shape[0]
print(NDVI)
dataSet = []
for i in range(1, n):
  a = list(map(float, NDVI[i][1:20]))
  dataSet.append(a)
band = range(len(NDVI[0][1:20]))
print(band)
# for i in range(1, n-1):
# plt.plot(band, dataSet[i], 'b-')

In [ ]:
from sklearn.cluster import KMeans
import joblib
import numpy
import time
import matplotlib.pyplot as plt
from google.colab import files

estimator = KMeans(n_clusters=4)#构造聚类器
estimator.fit(dataSet)#聚类
label_pred = estimator.labels_ #获取聚类标签
centroids = estimator.cluster_centers_ #获取聚类中心
inertia = estimator.inertia_ # 获取聚类准则的总和

result = estimator.fit_predict(dataSet)

print("Predicting result: ", result)
# print(len(dataSet))
numSamples = len(dataSet)
# print(centroids,type(centroids)) #显示中心点
# print(estimator.inertia_) #显示聚类效果
mark = ['-r', '-b', '-g', '-k', '^r', '+r', 'sr', 'dr', '<r', 'pr']

In [ ]:
# def Multipoin2feature(feature):
#   geomList = ee.Feature(feature).geometry().geometries();
#   def GetGeom(geom):
#     return ee.Feature(ee.Geometry(geom));
#   featureList = geomList.map(GetGeom)
#   return featureList;

# multiGeomExplodeList = cotton.toList(cotton.size()).map(Multipoin2feature).flatten();
# multiGeomExplodeFc = ee.FeatureCollection(multiGeomExplodeList);
# print(multiGeomExplodeFc.getInfo());

# size=multiGeomExplodeFc.size()
# toList = multiGeomExplodeFc.toList(size)
# idList = ee.List.sequence(0, size.subtract(1))
# model_list = ee.List(result.tolist())

# def add_property(x):
#   index = model_list.get(x)
#   feat = toList.get(x)
#   return ee.Feature(feat).set('model', index).copyProperties(feat)

# newList = idList.map(add_property)
# print(newList.getInfo(), 'newList')

In [ ]:
i = 0
a = 0
fig, ax = plt.subplots()
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
# plt.plot(band, centroids[i], '-b')
print("number = {}".format(a))
# print(corn_model_1_lon)
# print(corn_model_1_lat)
ax.set_xlabel('Time')
ax.set_ylabel('NDVI')
ax.set_title('Fruit Forest model 1')
plt.savefig('Fruit Forest model 1.svg',format='svg')
files.download('Fruit Forest model 1.svg')

In [ ]:
i = 1
a = 0
fig, ax = plt.subplots()
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
# plt.plot(band, centroids[i], '-b')
print("number = {}".format(a))
# print(corn_model_1_lon)
# print(corn_model_1_lat)
ax.set_xlabel('Time')
ax.set_ylabel('NDVI')
ax.set_title('Fruit Forest model 2')
plt.savefig('Fruit Forest model 2.svg',format='svg')
files.download('Fruit Forest model 2.svg')

In [ ]:
i = 2
a = 0
fig, ax = plt.subplots()
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
# plt.plot(band, centroids[i], '-b')
print("number = {}".format(a))
# print(corn_model_1_lon)
# print(corn_model_1_lat)
ax.set_xlabel('Time')
ax.set_ylabel('NDVI')
ax.set_title('Fruit Forest model 3')
plt.savefig('Fruit Forest model 3.svg',format='svg')
files.download('Fruit Forest model 3.svg')

In [ ]:
i = 3
a = 0
fig, ax = plt.subplots()
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
# plt.plot(band, centroids[i], '-b')
print("number = {}".format(a))
# print(corn_model_1_lon)
# print(corn_model_1_lat)
ax.set_xlabel('Time')
ax.set_ylabel('NDVI')
ax.set_title('Fruit Forest model 4')
plt.savefig('Fruit Forest model 4.svg',format='svg')
files.download('Fruit Forest model 4.svg')

In [ ]:
# geemap.ee_export_vector_to_asset(ee.FeatureCollection(newList), description='N_cotton', assetId="projects/ee-gaoliaoran2020/assets/N_cotton")

In [ ]:
# 打开矢量
csv_origin_dir = r"/content/gdrive/MyDrive/原始样本点NDVI时间序列"
os.chdir(csv_origin_dir)
CSV_list = glob.glob("*.csv")
print(CSV_list)
NDVI = []
i = 1
print(CSV_list[i])
with open(os.path.join(csv_origin_dir, CSV_list[i]), encoding='utf-8-sig') as f :
  for row in csv.reader(f, skipinitialspace=True):
    NDVI.append(row)
n = np.array(NDVI).shape[0]
dataSet = []
for i in range(1, n):
  a = list(map(float, NDVI[i][1:20]))
  dataSet.append(a)
band = range(len(NDVI[0][1:20]))
print(band)
# for i in range(1, n-1):
# plt.plot(band, dataSet[i], 'b-')

from sklearn.cluster import KMeans
import joblib
import numpy
import time
import matplotlib.pyplot as plt

estimator = KMeans(n_clusters=4)#构造聚类器
estimator.fit(dataSet)#聚类
label_pred = estimator.labels_ #获取聚类标签
centroids = estimator.cluster_centers_ #获取聚类中心
inertia = estimator.inertia_ # 获取聚类准则的总和

result = estimator.fit_predict(dataSet)

print("Predicting result: ", result)
# print(len(dataSet))
numSamples = len(dataSet)
# print(centroids,type(centroids)) #显示中心点
# print(estimator.inertia_) #显示聚类效果
mark = ['-r', '-b', '-g', '-k', '^r', '+r', 'sr', 'dr', '<r', 'pr']

In [ ]:
# def Multipoin2feature(feature):
#   geomList = ee.Feature(feature).geometry().geometries();
#   def GetGeom(geom):
#     return ee.Feature(ee.Geometry(geom));
#   featureList = geomList.map(GetGeom)
#   return featureList;

# multiGeomExplodeList = wheat.toList(wheat.size()).map(Multipoin2feature).flatten();
# multiGeomExplodeFc = ee.FeatureCollection(multiGeomExplodeList);
# print(multiGeomExplodeFc.getInfo());

# size=multiGeomExplodeFc.size()
# toList = multiGeomExplodeFc.toList(size)
# idList = ee.List.sequence(0, size.subtract(1))
# model_list = ee.List(result.tolist())

# def add_property(x):
#   index = model_list.get(x)
#   feat = toList.get(x)
#   return ee.Feature(feat).set('model', index).copyProperties(feat)

# newList = idList.map(add_property)
# print(newList.getInfo(), 'newList')

In [ ]:
i = 0
a = 0
fig, ax = plt.subplots()
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
# plt.plot(band, centroids[i], '-b')
print("number = {}".format(a))
# print(corn_model_1_lon)
# print(corn_model_1_lat)
ax.set_xlabel('Time')
ax.set_ylabel('NDVI')
ax.set_title('Wheat model 1')
plt.savefig('Wheat model 1.svg',format='svg')
files.download('Wheat model 1.svg')

In [ ]:
i = 1
a = 0
fig, ax = plt.subplots()
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
# plt.plot(band, centroids[i], '-b')
print("number = {}".format(a))
# print(corn_model_1_lon)
# print(corn_model_1_lat)
ax.set_xlabel('Time')
ax.set_ylabel('NDVI')
ax.set_title('Wheat model 2')
plt.savefig('Wheat model 2.svg',format='svg')
files.download('Wheat model 2.svg')

In [ ]:
i = 2
a = 0
fig, ax = plt.subplots()
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
# plt.plot(band, centroids[i], '-b')
print("number = {}".format(a))
# print(corn_model_1_lon)
# print(corn_model_1_lat)
ax.set_xlabel('Time')
ax.set_ylabel('NDVI')
ax.set_title('Wheat model 3')
plt.savefig('Wheat model 3.svg',format='svg')
files.download('Wheat model 3.svg')

In [ ]:
i = 3
a = 0
fig, ax = plt.subplots()
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
# plt.plot(band, centroids[i], '-b')
print("number = {}".format(a))
# print(corn_model_1_lon)
# print(corn_model_1_lat)
ax.set_xlabel('Time')
ax.set_ylabel('NDVI')
ax.set_title('Wheat model 4')
plt.savefig('Wheat model 4.svg',format='svg')
files.download('Wheat model 4.svg')

In [ ]:
# geemap.ee_export_vector_to_asset(ee.FeatureCollection(newList), description='N_wheat', assetId="projects/ee-gaoliaoran2020/assets/N_wheat")

In [ ]:
# 打开矢量
csv_origin_dir = r"/content/gdrive/MyDrive/原始样本点NDVI时间序列"
os.chdir(csv_origin_dir)
CSV_list = glob.glob("*.csv")
print(CSV_list)
NDVI = []
i = 2
print(CSV_list[i])
with open(os.path.join(csv_origin_dir, CSV_list[i]), encoding='utf-8-sig') as f :
  for row in csv.reader(f, skipinitialspace=True):
    NDVI.append(row)
n = np.array(NDVI).shape[0]
dataSet = []
for i in range(1, n):
  a = list(map(float, NDVI[i][1:20]))
  dataSet.append(a)
band = range(len(NDVI[0][1:20]))
print(band)
# for i in range(1, n-1):
# plt.plot(band, dataSet[i], 'b-')

from sklearn.cluster import KMeans
import joblib
import numpy
import time
import matplotlib.pyplot as plt

estimator = KMeans(n_clusters=3)#构造聚类器
estimator.fit(dataSet)#聚类
label_pred = estimator.labels_ #获取聚类标签
centroids = estimator.cluster_centers_ #获取聚类中心
inertia = estimator.inertia_ # 获取聚类准则的总和

result = estimator.fit_predict(dataSet)

print("Predicting result: ", result)
# print(len(dataSet))
numSamples = len(dataSet)
# print(centroids,type(centroids)) #显示中心点
# print(estimator.inertia_) #显示聚类效果
mark = ['-r', '-b', '-g', '-k', '^r', '+r', 'sr', 'dr', '<r', 'pr']

In [ ]:
# def Multipoin2feature(feature):
#   geomList = ee.Feature(feature).geometry().geometries();
#   def GetGeom(geom):
#     return ee.Feature(ee.Geometry(geom));
#   featureList = geomList.map(GetGeom)
#   return featureList;
# # --------------------------------------------------------------------------------------------------
# multiGeomExplodeList = forest.toList(forest.size()).map(Multipoin2feature).flatten();
# # --------------------------------------------------------------------------------------------------
# multiGeomExplodeFc = ee.FeatureCollection(multiGeomExplodeList);
# print(multiGeomExplodeFc.getInfo());

# size=multiGeomExplodeFc.size()
# toList = multiGeomExplodeFc.toList(size)
# idList = ee.List.sequence(0, size.subtract(1))
# model_list = ee.List(result.tolist())

# def add_property(x):
#   index = model_list.get(x)
#   feat = toList.get(x)
#   return ee.Feature(feat).set('model', index).copyProperties(feat)

# newList = idList.map(add_property)
# print(newList.getInfo(), 'newList')

In [ ]:
i = 0
a = 0
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
print("number = {}".format(a))

In [ ]:
i = 1
a = 0
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
print("number = {}".format(a))

In [ ]:
i = 2
a = 0
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
print("number = {}".format(a))

In [ ]:
# # --------------------------------------------------------------------------------------------------
# geemap.ee_export_vector_to_asset(ee.FeatureCollection(newList), description='N_forest', assetId="projects/ee-gaoliaoran2020/assets/N_forest")
# # --------------------------------------------------------------------------------------------------

In [ ]:
# 打开矢量
csv_origin_dir = r"/content/drive/MyDrive/原始样本点NDVI时间序列"
os.chdir(csv_origin_dir)
CSV_list = glob.glob("*.csv")
print(CSV_list)
NDVI = []
i = 2
print(CSV_list[i])
with open(os.path.join(csv_origin_dir, CSV_list[i]), encoding='utf-8-sig') as f :
  for row in csv.reader(f, skipinitialspace=True):
    NDVI.append(row)
n = np.array(NDVI).shape[0]
dataSet = []
for i in range(1, n):
  a = list(map(float, NDVI[i][1:20]))
  dataSet.append(a)
band = range(len(NDVI[0][1:20]))
print(band)
# for i in range(1, n-1):
# plt.plot(band, dataSet[i], 'b-')

from sklearn.cluster import KMeans
import joblib
import numpy
import time
import matplotlib.pyplot as plt

estimator = KMeans(n_clusters=3)#构造聚类器
estimator.fit(dataSet)#聚类
label_pred = estimator.labels_ #获取聚类标签
centroids = estimator.cluster_centers_ #获取聚类中心
inertia = estimator.inertia_ # 获取聚类准则的总和

result = estimator.fit_predict(dataSet)

print("Predicting result: ", result)
# print(len(dataSet))
numSamples = len(dataSet)
# print(centroids,type(centroids)) #显示中心点
# print(estimator.inertia_) #显示聚类效果
mark = ['-r', '-b', '-g', '-k', '^r', '+r', 'sr', 'dr', '<r', 'pr']

In [ ]:
# def Multipoin2feature(feature):
#   geomList = ee.Feature(feature).geometry().geometries();
#   def GetGeom(geom):
#     return ee.Feature(ee.Geometry(geom));
#   featureList = geomList.map(GetGeom)
#   return featureList;
# # --------------------------------------------------------------------------------------------------
# multiGeomExplodeList = corn.toList(corn.size()).map(Multipoin2feature).flatten();
# # --------------------------------------------------------------------------------------------------
# multiGeomExplodeFc = ee.FeatureCollection(multiGeomExplodeList);
# print(multiGeomExplodeFc.getInfo());

# size=multiGeomExplodeFc.size()
# toList = multiGeomExplodeFc.toList(size)
# idList = ee.List.sequence(0, size.subtract(1))
# model_list = ee.List(result.tolist())

# def add_property(x):
#   index = model_list.get(x)
#   feat = toList.get(x)
#   return ee.Feature(feat).set('model', index).copyProperties(feat)

# newList = idList.map(add_property)
# print(newList.getInfo(), 'newList')

In [ ]:
# # --------------------------------------------------------------------------------------------------
# geemap.ee_export_vector_to_asset(ee.FeatureCollection(newList), description='N_corn', assetId="projects/ee-gaoliaoran2020/assets/N_corn")
# # --------------------------------------------------------------------------------------------------

In [ ]:
print(centroids,type(centroids)) #显示中心点

In [ ]:
i = 0
a = 0
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
print("number = {}".format(a))

In [ ]:
i = 1
a = 0
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
print("number = {}".format(a))

In [ ]:
i = 2
a = 0
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
print("number = {}".format(a))

In [ ]:
# 打开矢量
csv_origin_dir = r"/content/gdrive/MyDrive/原始样本点NDVI时间序列"
os.chdir(csv_origin_dir)
CSV_list = glob.glob("*.csv")
print(CSV_list)
NDVI = []
i = 4
print(CSV_list[i])
with open(os.path.join(csv_origin_dir, CSV_list[i]), encoding='utf-8-sig') as f :
  for row in csv.reader(f, skipinitialspace=True):
    NDVI.append(row)
n = np.array(NDVI).shape[0]
dataSet = []
for i in range(1, n):
  a = list(map(float, NDVI[i][1:20]))
  dataSet.append(a)
band = range(len(NDVI[0][1:20]))
print(band)
# for i in range(1, n-1):
# plt.plot(band, dataSet[i], 'b-')

from sklearn.cluster import KMeans
import joblib
import numpy
import time
import matplotlib.pyplot as plt

estimator = KMeans(n_clusters=3)#构造聚类器
estimator.fit(dataSet)#聚类
label_pred = estimator.labels_ #获取聚类标签
centroids = estimator.cluster_centers_ #获取聚类中心
inertia = estimator.inertia_ # 获取聚类准则的总和

result = estimator.fit_predict(dataSet)

print("Predicting result: ", result)
# print(len(dataSet))
numSamples = len(dataSet)
# print(centroids,type(centroids)) #显示中心点
# print(estimator.inertia_) #显示聚类效果
mark = ['-r', '-b', '-g', '-k', '^r', '+r', 'sr', 'dr', '<r', 'pr']

In [ ]:
# def Multipoin2feature(feature):
#   geomList = ee.Feature(feature).geometry().geometries();
#   def GetGeom(geom):
#     return ee.Feature(ee.Geometry(geom));
#   featureList = geomList.map(GetGeom)
#   return featureList;

# multiGeomExplodeList = wheat_corn.toList(wheat_corn.size()).map(Multipoin2feature).flatten();
# multiGeomExplodeFc = ee.FeatureCollection(multiGeomExplodeList);
# print(multiGeomExplodeFc.getInfo());

# size=multiGeomExplodeFc.size()
# toList = multiGeomExplodeFc.toList(size)
# idList = ee.List.sequence(0, size.subtract(1))
# model_list = ee.List(result.tolist())

# def add_property(x):
#   index = model_list.get(x)
#   feat = toList.get(x)
#   return ee.Feature(feat).set('model', index).copyProperties(feat)

# newList = idList.map(add_property)
# print(newList.getInfo(), 'newList')

In [ ]:
# geemap.ee_export_vector_to_asset(ee.FeatureCollection(newList), description='N_wheat_corn', assetId="projects/ee-gaoliaoran2020/assets/N_wheat_corn")

In [ ]:
i = 0
a = 0
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
print("number = {}".format(a))

In [ ]:
i = 1
a = 0
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
print("number = {}".format(a))

In [ ]:
i = 2
a = 0
for j in range(1, len(result)):
  if result[j] == i:
    a = a+1
    plt.plot(band, dataSet[j], 'b-')
print("number = {}".format(a))